# Experimental analysis

This notebook provides the software required to perform the experimentation that accompanies the following article:

    A Genetic Clustering Approach to Extract Data from HTML Tables
	Patricia Jiménez, Juan C. Roldán, Rafael Corchuelo
	Submitted to Information Processing & Management

Prior to running it, you must create a conda environment that hosts a Python 3.7 interpreter and then install the dependencies in file "requirements.txt".

Once your environment is ready, please, go ahead!  Just execute the following cells one after the other.


## Initialisation

Please, run the following cell to set the experimentation environment up.  Don't worry if you get some deprecation warnings regarding TensorFlow. 

In [ ]:
from attention_decoder import AttentionDecoder
from bs4 import BeautifulSoup as soup
from keras import Sequential
from keras.layers import Reshape, LSTM, Convolution2D, BatchNormalization, Dense, Flatten
from numpy import array, argmax
from os import listdir
from pickle import load
from pprint import pprint
from random import sample, seed, shuffle
from regex import sub
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score
from statistics import stdev, mean

from tablextract import segmentate, functional_analysis, structural_analysis
from tablextract_melva import segmentate as melva_segmentate, functional_analysis as melva_functional_analysis, structural_analysis as melva_structural_analysis

from time import process_time as cpu_time
from utils import *

Let's now set a few configuration parameters.

In [ ]:
# Set "SAMPLE" to True to perform a quick experimentation on a small subset of tables.
SAMPLE = False

# Set "FOLDS" to the number of folds to used during k-fold cross validation.
FOLDS = 3

# Set "SEED" to an arbitrary string to reproduce the results.  None means total randomness.

#SEED = 'TOMATE301219'
SEED = None

# Set "PATH_RESULTS" to the CSV file where the results will be output.  
# Include a "%s" that will be replaced by the name of each proposal at runtime
PATH_RESULTS = './output/results_%s.csv'

Let's now load the tables and compute their features.  For performance purposes, we only keep the features used by the proposals that are compared in this experimental analysis.

In [ ]:
tables = load_tables(function_names=FUNCTION_NAMES_EXPERIMENTATION)
keep_features = ('tag', 'colspan', 'rowspan', 'relative-row', 'relative-col', 'background-color-r', 'background-color-g', 'background-color-b', 'font-family')
for table in tables:
    with open(PATH_ORIGINAL_TABLE % table['_id'], 'rb') as fp:
        table['features'] = [
            [
                {
                    feat: cell[feat]
                    for feat in keep_features
                    if feat in cell
                }
                for cell in row
            ] for row in load(fp).features
        ]
init_num_tables = len(tables)
tables = [
    t for t in tables
    if len([cell for row in t['features'] for cell in row]) == len([cell for row in t['functions'] for cell in row]) == len([cell for row in t['texts'] for cell in row])
]
final_num_tables = len(tables)

print('There are %s annotated tables in the repository.' % init_num_tables)
print('There are %s fully-annotated tables.' % final_num_tables)
print('A total of %s partially-annotated tables were ignored.' % (init_num_tables - final_num_tables))

Finally, let's shuffle the tables, split the datasets into Wikipedia and DWTC, and split them into folds.

In [ ]:
seed(SEED)
shuffle(tables)

if SAMPLE:
    corpuses = [t for t in tables if t['kind'] == 'horizontal listing'][:40]
    corpuses.extend([t for t in tables if t['kind'] == 'vertical listing'][:30])
    corpuses.extend([t for t in tables if t['kind'] == 'matrix'][:30])
    corpuses = {'Sample': corpuses}
else:    
    corpuses = {
        'Wikipedia': [t for t in tables if 'en.wikipedia' in t['document_id']],
        'DWTC': [t for t in tables if 'en.wikipedia' not in t['document_id']]
    }

print('Tables by dataset:\n%s' % '\n'.join('   %s: %s' % (k, len(v)) for k, v in corpuses.items()))
print('Cells by dataset:\n%s' % '\n'.join('   %s: %s' % (k, sum(sum(len(row) for row in t) for t in v)) for k, v in corpuses.items()))

for c in corpuses:
    fold_size = int(len(corpuses[c]) / FOLDS)
    corpuses[c] = [corpuses[c][n:n + fold_size] for n in range(0, len(corpuses[c]), fold_size)]

## Evaluation method

This method performs the evaluation.  Given a proposal to evaluate (extractor), it runs it and computes the experimental results, namely: precision, recall, F_1, average number and deviation of CPU second per table.  The measures are computed using k-fold-cross validation.  (The value of k is taken from variable "FOLDS", which was set above.)

In [ ]:
def evaluate(extractor, label, multitable_prediction=False):
    rows = [['CORPUS', 'TABLE_ID', 'TABLE_KIND', 'P', 'R', 'F_1', 'ACCURACY', 'TRAIN_TIME', 'PREDICT_TIME']]
    for c in corpuses:
        print(c)
        for f in range(FOLDS):
            print(f"Fold #{f + 1}/{FOLDS}")
            # get the train and test folds
            train = [table for n, fold in enumerate(corpuses[c]) for table in fold if n != f]
            test = corpuses[c][f]
            # instantiate and train the extractor
            ext = extractor()
            seed(SEED)
            train_time = cpu_time()            
            ext.train(train)            
            train_time = (cpu_time() - train_time) / len(train)
            # test the extractor
            if multitable_prediction:
                seed(SEED)
                predict_time = cpu_time()                
                predicted_tables, discount_time = ext.predict(test)                
                predict_time = (cpu_time() - predict_time - discount_time) / len(test)
                for table, predicted_table in zip(test, predicted_tables):
                    annotated = [cell for row in table['functions'] for cell in row]
                    predicted = [cell for row in predicted_table for cell in row]
                    #if len(set(predicted)) == 1: print(f"ONLY ONE CLUSTER {table['_id']}")
                    p, r, f_1, _ = precision_recall_fscore_support(annotated, predicted, average='macro', labels=('data', 'meta-data'), zero_division=0)
                    accuracy = accuracy_score(annotated, predicted)
                    #roc_auc = roc_auc_score(annotated, predicted, average='macro')
                    rows.append([c, table['_id'], table['kind'], p, r, f_1, accuracy, train_time, predict_time])
            else:
                for table in test:
                    annotated = [cell for row in table['functions'] for cell in row]
                    predict_time = cpu_time()
                    try:
                        predicted_table, discount_time = ext.predict(table)
                        predicted = [cell for row in predicted_table for cell in row]
                    except:
                        print(f"EXCEPTION OCCURRED TABLE {table['_id']}")
                        print_exc()
                        discount_time = 0
                        predicted = ['failed' for _ in range(len(annotated))]
                    predict_time = cpu_time() - predict_time - discount_time
                    #if len(set(predicted)) == 1: print(f"ONLY ONE CLUSTER {table['_id']}")
                    p, r, f_1, _ = precision_recall_fscore_support(annotated, predicted, average='macro', labels=('data', 'meta-data'), zero_division=0)
                    accuracy = accuracy_score(annotated, predicted)
                    #roc_auc = roc_auc_score(annotated, predicted, average='macro')
                    rows.append([c, table['_id'], table['kind'], p, r, f_1, accuracy, train_time, predict_time])
    # export the evaluation result
    with open(PATH_RESULTS % label, 'w') as fp:
        fp.write('\n'.join(
            '\t'.join(map(str, row))
            for row in rows
        ))
    # print summary results
    for c in corpuses:
        print(c)
        corpus_rows = [row for row in rows[1:] if row[0] == c]
        for kind in ('overall', 'horizontal listing', 'vertical listing', 'matrix'):
            kind_rows = [row for row in corpus_rows if row[2] == kind or kind == 'overall']
            p = mean([row[3] for row in kind_rows])
            r = mean([row[4] for row in kind_rows])
            f_1 = mean([row[5] for row in kind_rows])
            accuracy = mean([row[6] for row in kind_rows])
            time_avg = mean([row[7] + row[8] for row in kind_rows])
            time_dev = stdev([row[7] + row[8] for row in kind_rows])
            print(
                '\t%s: p = %.2f, r = %.2f, f_1 = %.2f, acc = %.2f, time_avg = %.2f, time_dev = %.2f'
                %
                (kind.rjust(25), 100 * p, 100 * r, 100 * f_1, 100 * accuracy, time_avg, time_dev)
            )

## Yoshida et al.

The following cell implements Yoshida et al.'s proposal and evaluates it.  Note that the cell produces some output during the evaluation.  The results are stored in the file specified by variable "PATH_RESULTS", which was defined above.

In [ ]:
class YoshidaExtractor:
    
    layouts = ((0, None), (1, 'h'), (1, 'v'), (2, 'h'), (3, 'h'), (4, 'h'), (2, 'v'), (3, 'v'), (4, 'v'))

    def train(self, tables):
        pass

    def predict(self, tables):
        knds = self.kinds(tables)
        res = []
        for table, (period, orientation) in zip(tables, knds):
            pred_table = [
                [
                    'data' if self.get_function(period, orientation, r, c) == 0 else 'meta-data'
                    for c in range(len(row))
                ] for r, row in enumerate(table['texts'])
            ]
            res.append(pred_table)
        return res, 0

    def kinds(self, tables, iterations=10, repetitions=100):
        T = [table['texts'] for table in tables]
        best_score = -1
        best_M = None
        for _ in range(repetitions):
            M = [sample(self.layouts, 1)[0] for _ in range(len(tables))]
            for i in range(iterations):
                probs = self.compute_probs(T, M)
                M, score = self.compute_layouts(T, probs)
            if score > best_score:
                best_score = score
                best_M = M
        return best_M

    def get_function(self, period, orientation, r, c):
        if orientation == None:
            res = 0
        else:
            i = r if orientation == 'h' else c
            if period == 1:
                res = 1 if r == 0 else 0
            elif period == 4:
                res = 1 if r == 1 else 0
            else:
                res = 1 if r % period == 0 else 0
        return res

    def compute_probs(self, T, M):
        ''' Given a list of tables and their type, return a dictionary with cell
        text as keys and probability of them being data as values. '''
        def add_occurrence(dct, text, kind):
            if text not in dct: dct[text] = [0, 0]
            dct[text][kind] += 1
        res = {}
        for table, (period, orientation) in zip(T, M):
            for r, row in enumerate(table):
                for c, cell in enumerate(row):
                    add_occurrence(res, cell, self.get_function(period, orientation, r, c))
        return {k: (v[0] / (v[0] + v[1]), v[1] / (v[0] + v[1])) for k, v in res.items()}

    def compute_layouts(self, T, probs):
        res = []
        total_score = 0
        for table in T:
            max_score = -1
            max_type = None
            for period, orientation in self.layouts:
                score = 0
                for r, row in enumerate(table):
                    for c, cell in enumerate(row):
                        kind = self.get_function(period, orientation, r, c)
                        score += probs[cell][kind]
                if score > max_score:
                    max_score = score
                    max_type = (period, orientation)
            res.append(max_type)
            total_score += max_score / (len(table) * len(table[0]))
        return res, total_score

evaluate(YoshidaExtractor, 'yoshida', multitable_prediction=True)

## Embley


The following cell implements Embley et al.'s proposal and evaluates it. Note that the cell produces some output during the evaluation.  The results are stored in the file specified by variable "PATH_RESULTS", which was defined above.

In [ ]:
class EmbleyExtractor:
    def train(self, tables):
        pass

    def predict(self, table):
        res = self.functions(table['texts'])
        return [['data' if cell == 0 else 'meta-data' for cell in row] for row in res], 0
        
    def functions(self, table):
        CC1, CC2 = self.mips(table)
        res = []
        for r, row in enumerate(table, 1):
            res.append([])
            for c, cell in enumerate(row, 1):
                if CC1[0] <= r <= CC2[0] or CC1[1] <= c <= CC2[1]:
                    res[-1].append(1)
                else:
                    res[-1].append(0)
        return res

    def no_duplicates(self, table, r1, c1, r2, c2, kind='rows', empty_matters=False):
        tab = [row[c1 - 1: c2] for row in table[r1 - 1: r2]]
        if empty_matters: return False
        if kind == 'cols': tab = [*zip(*tab)]  # tricky list trasposition
        tab = [tuple(row) for row in tab]
        return len(tab) == len(set(tab))

    def mips(self, table):
        # initialize
        CC1, CC2 = None, (1, 1)
        C_max, R_max = len(table[0]), len(table)  # CC4 is trivial in web tables
        R_1 = 1; C_1 = 1; R_2 = R_max - 1; C_2 = 1
        rightflag = upflag = 0
        max_area = 0

        # locate candidate MIPs by finding the minimum indexing headers
        while C_2 < C_max and R_2 >= R_1:
            if self.no_duplicates(table, R_2 + 1, C_1, R_max, C_2, 'rows') and self.no_duplicates(table, R_1, C_2 + 1, R_2 - 1, C_max, 'cols'):
                R_2 -= 1
                upflag = 1; rightflag = 0
            else:
                C_2 += 1
                rightflag = 1
                if upflag == rightflag == 1:
                    data_area = (R_max - R_2 + 1) * (C_max - C_2 + 1)
                    if data_area > max_area:
                        max_area = data_area
                        CC2 = (R_2, C_2)
                    upflag = 0

        # locate CC1 at intersection of the top row and the leftmost column necessary for indexing
        R_1, C_1 = 1, 1
        while self.no_duplicates(table, R_1 + 1, C_2 + 1, R_2, C_max, 'cols', True): R_1 += 1
        while self.no_duplicates(table, R_2 + 1, C_1 + 1, C_2, R_max, 'rows', True): C_1 += 1
        return (R_1, C_1), CC2

evaluate(EmbleyExtractor, 'embley')

## Jung and Kwon

The following cell implements Jung and Kwon's proposal and evaluates it. Note that the cell produces some output during the evaluation.  The results are stored in the file specified by variable "PATH_RESULTS", which was defined above.

In [ ]:
class JungExtractor:
    def train(self, tables):
        pass
    
    def predict(self, table):
        res = self.functions(table)
        return [['data' if cell == 0 else 'meta-data' for cell in row] for row in res], 0
        
    def functions(self, table):
        w = len(table['texts'][0])
        h = len(table['texts'])
        heurs = [[[-1 for c in range(w)] for r in range(h)] for _ in range(7)]

        # cell property heuristics: 5.1 and 5.6
        for r, (row_text, row_feats) in enumerate(zip(table['texts'], table['features'])):
            for c, (cell_text, cell_feats) in enumerate(zip(row_text, row_feats)):
                if 'tag' in cell_feats:
                    # heuristic 5.1
                    heurs[0][r][c] = int(cell_feats['tag'] == 'th')
                    # heuristic 5.6
                    heurs[5][r][c] = int((cell_feats['colspan'] * w > 1 or cell_feats['rowspan'] * h > 1) and (cell_feats['relative-row'] == 0 or cell_feats['relative-col'] == 0))
                else:
                    heurs[0][r][c] = 0
                    heurs[5][r][c] = 0

        # heuristics 5.2
        bgcols = [
            [
                (int(255 * cell['background-color-r']), int(255 * cell['background-color-g']), int(255 * cell['background-color-b'])) if 'tag' in cell else 0
                for cell in row
            ] for row in table['features']
        ]
        palette = list(set(bgcol for row in bgcols for bgcol in row))
        if len(palette) == 2:
            pos = {p: 0 for p in palette}
            for r in range(h):
                for c in range(w):
                    pos[bgcols[r][c]] += c + r
            header_color = min(palette, key=lambda x: pos[x])
            for r in range(h):
                for c in range(w):
                    heurs[1][r][c] = 1 if bgcols[r][c] == header_color else 0

        # heuristics 5.3
        bgcols = [
            [
                cell['font-family']
                for cell in row if 'tag' in cell
            ] for row in table['features']
        ]
        palette = list(set(bgcol for row in bgcols for bgcol in row))
        if len(palette) == 2:
            pos = {p: 0 for p in palette}
            for r in range(min(h, len(bgcols))):
                for c in range(min(w, len(bgcols[r]))):
                    pos[bgcols[r][c]] += c + r
            header_color = min(palette, key=lambda x: pos[x])
            for r in range(min(h, len(bgcols))):
                for c in range(min(w, len(bgcols[r]))):
                    heurs[1][r][c] = 1 if bgcols[r][c] == header_color else 0

        # heuristic 5.5
        def patternise(text):
            return sub(r'\(.+\)', 'I', sub(r'\d+', 'D', sub(r'[\p{L}]+', 'W', text)))

        for r in range(h):
            row = [patternise(c) for c in table['texts'][r]]
            if len(set(row[1:])) == 1:
                heurs[4][r][0] = 1

        for c in range(w):
            col = [patternise(row[c]) for row in table['texts']]
            if len(set(col[1:])) == 1:
                heurs[4][0][c] = 1

        if any(c == 1 for row in heurs[4] for c in row):
            for r in range(h):
                for c in range(w):
                    if heurs[4][r][c] == -1: heurs[4][r][c] = 0

        # table heuristics: 5.7
        if any(len(t) == 0 for t in table['texts'][0]):
            for c in range(w): heurs[6][0][c] = 1
        if any(len(row[0]) == 0 for row in table['texts']):
            for r in range(h): heurs[6][r][0] = 1
        if any(c == 1 for row in heurs[6] for c in row):
            for r in range(h):
                for c in range(w):
                    if heurs[6][r][c] == -1: heurs[6][r][c] = 0

        weights = [.125, .188, .125, .125, .0625, .1875, .187]
        to_ignore = []
        for n, heur in enumerate(heurs):
            if heur[0][0] == -1: to_ignore.append(n)
        for n in reversed(to_ignore):
            weights.pop(n)
            heurs.pop(n)

        weights = [w / sum(weights) for w in weights]
        functions = [[int(round(sum(w * heur[r][c] for w, heur in zip(weights, heurs)))) for c in range(w)] for r in range(h)]
        return functions

evaluate(JungExtractor, 'jung')

## Nishida et al.

The following cell implements Nishida et al.'s proposal and evaluates it. Note that the cell produces some output during the evaluation.  The results are stored in the file specified by variable "PATH_RESULTS", which was defined above.  Please, note that this proposal relies on TensorFlow, which issues several deprecation warnings that you may safely ignore.

In [ ]:
class NishidaExtractor:
    def __init__(self):
        model = Sequential()
        model.add(Reshape((64, 50 * 100)))
        model.add(LSTM(100, return_sequences=True))
        model.add(AttentionDecoder(100, 100))
        model.add(Reshape((8, 8, 100)))
        model.add(Convolution2D(32, (3, 3)))
        model.add(BatchNormalization())
        model.add(Convolution2D(32, (3, 3)))
        model.add(BatchNormalization())
        model.add(Flatten())
        model.add(Dense(3))
        model.compile(optimizer='SGD', loss='mean_squared_error')
        self.model = model
        self.batch_size = 5 if SAMPLE else 50
        self.epochs = 8 if SAMPLE else 80

    def train(self, tables):
        ids = [NISHIDA_IDS.index(t['_id']) for t in tables]
        self.model.fit(x=NISHIDA_X[ids], y=NISHIDA_Y[ids], batch_size=self.batch_size, epochs=self.epochs)

    def predict(self, table):
        rows, cols = len(table['texts']), len(table['texts'][0])
        tab = NISHIDA_X[NISHIDA_IDS.index(table['_id'])]
        cls = self.model.predict(array([tab]))
        cls = argmax(cls)
        res = [['data' for _ in range(cols)] for _ in range(rows)]
        if cls == 0 or cls == 2:
            res[0] = ['meta-data' for _ in range(cols)]
        if cls == 1 or cls == 2:
            for r in range(rows):
                res[r][0] = 'meta-data'
        return res, 0

with open('../DATA/nishida/names.pk', 'rb') as fp:
    NISHIDA_IDS = load(fp)
with open('../DATA/nishida/tables.pk', 'rb') as fp:
    NISHIDA_X = load(fp)
with open('../DATA/nishida/kinds.pk', 'rb') as fp:
    NISHIDA_Y = load(fp)
    
evaluate(NishidaExtractor, 'nishida')

# Melva

The following cell implements our proposal and evaluates it. Note that the cell produces some output during the evaluation.  The results are stored in the file specified by variable "PATH_RESULTS", which was defined above.

In [ ]:
from WorkerManager import WorkerManager
from DiskCache import DiskCache

WorkerManager.initialise()

class TomateMelvaExtractor:
    def __init__(
        self,
        normalization='min-max-global',  # The choices are "min-max-global", "min-max-local", "standard", or "softmax"
        clustering_features=['style', 'syntax', 'structural', 'semantic'],  # any subset of these values is valid for experimentation.
        dimensionality_reduction='off',  # "off" is the only choice with Melva, since melva implements its own dim. red. procedure.
        clustering_method='melva' # "melva" is the only choice now.
    ):
        self.normalization = normalization
        self.clustering_features = clustering_features
        self.dimensionality_reduction = dimensionality_reduction
        self.clustering_method = clustering_method
        
    def train(self, tables):
        DiskCache.initialise()        
    
    def predict(self, table):
        DiskCache.initialise()
        discount_time = cpu_time()
        with open(PATH_ORIGINAL_TABLE % table['_id'], 'rb') as fp:
            source = load(fp)
        source.element = soup(source.element)
        melva_segmentate(source, add_image_text=True, add_link_urls=False, base_url=source.url, normalization=self.normalization, text_metadata_dict=METADATA_CORPUS)
        discount_time = cpu_time() - discount_time
        
        melva_functional_analysis(source, self.clustering_features, self.dimensionality_reduction, self.clustering_method)
        melva_structural_analysis(source)
        predicted_table = [[FUNCTION_NAMES_PREDICTION[cell] for cell in row] for row in source.functions]
        
        return predicted_table, discount_time

evaluate(TomateMelvaExtractor, "melva")